In [1]:
import sys
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

#sys.path.insert(0, "/usr/src/app/app")
def import_path(fullpath):
    """ 
    Import a file with full path specification. Allows one to
    import from anywhere, something __import__ does not do. 
    """
    path, filename = os.path.split(fullpath)
    filename, ext = os.path.splitext(filename)
    sys.path.append(path)
    module = __import__(filename)
   # reload(module) # Might be out of date
    del sys.path[-1]
    return module
import_path("/usr/src/app/app")

<module 'app' from '/usr/src/app/app/__init__.py'>

In [2]:
from PIL import Image

from app.pkg.ml.try_on.preprocessing.aggregator import ClothProcessor
from app.pkg.ml.try_on.preprocessing.aggregator import HumanProcessor
from app.pkg.ml.try_on.lady_vton import LadyVtonAggregator
from app.pkg.ml.try_on.postprocessing.fix_face import FaceFixer
from app.pkg.models.app.image_category import ImageCategory
from app.pkg.ml.buffer_converters import BytesConverter
bc = BytesConverter()


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
human_model = HumanProcessor()
cloth_model = ClothProcessor()
try_on_model = LadyVtonAggregator()


Some weights of the model checkpoint at laion/CLIP-ViT-H-14-laion2B-s32B-b79K were not used when initializing CLIPVisionModelWithProjection: ['text_model.encoder.layers.15.mlp.fc2.bias', 'text_model.encoder.layers.6.self_attn.k_proj.bias', 'text_model.encoder.layers.16.layer_norm2.bias', 'text_model.encoder.layers.22.self_attn.v_proj.bias', 'text_model.encoder.layers.4.self_attn.q_proj.weight', 'text_model.encoder.layers.2.mlp.fc1.weight', 'text_model.encoder.layers.16.self_attn.k_proj.bias', 'text_model.encoder.layers.23.self_attn.q_proj.weight', 'text_model.encoder.layers.19.self_attn.out_proj.bias', 'text_model.encoder.layers.1.layer_norm1.weight', 'text_model.encoder.layers.22.self_attn.k_proj.bias', 'text_model.encoder.layers.10.self_attn.q_proj.weight', 'text_model.encoder.layers.6.layer_norm1.bias', 'text_model.encoder.layers.9.self_attn.out_proj.bias', 'text_model.encoder.layers.8.mlp.fc2.bias', 'text_model.encoder.layers.8.self_attn.v_proj.bias', 'text_model.encoder.layers.23.

initialization method [normal]
initialization method [normal]


2024-04-24 16:10:34,785 - [INFO] - app.pkg.ml.try_on.postprocessing.fix_face - (fix_face.py).__init__(13) - FaceFixer inited 


In [4]:

# cloth_fp = "/usr/src/app/data/example/white_pants.png"
clothes = [   

    {"fp":"/usr/src/app/data/lower/b_black_jeans.png", "category":ImageCategory.LOWER_BODY},
    {"fp": "/usr/src/app/data/lower/jeans-dc.png", "category":ImageCategory.LOWER_BODY},
    {"fp": "/usr/src/app/data/lower/b_shorts.png", "category":ImageCategory.LOWER_BODY},
    {"fp": "/usr/src/app/data/lower/shorts-dc.png", "category":ImageCategory.LOWER_BODY},
    {"fp": "/usr/src/app/data/lower/jeans-dc.png", "category":ImageCategory.LOWER_BODY},
    {"fp":"/usr/src/app/data/dresses/2.png", "category":ImageCategory.DRESSES},

    {"fp":"/usr/src/app/data/lower/b_black_jeans.png", "category":ImageCategory.LOWER_BODY},
    {"fp": "/usr/src/app/data/lower/jeans-dc.png", "category":ImageCategory.LOWER_BODY},
]
human_fp = "/usr/src/app/data/human/jennifer_lourence.png"

In [5]:

user_image = Image.open(human_fp)
for cloth in clothes:
    cloth_image = Image.open(cloth['fp'])
    no_background_image = cloth_model.model_background(cloth_image)
    no_background_image_bytes = bc.image_to_bytes(no_background_image)
    cloth['cloth'] = no_background_image_bytes



# no_background_image = cloth_model.model_background(cloth_image)
# no_background_image_bytes = bc.image_to_bytes(no_background_image)
# result = {}
# result["cloth"] = no_background_image_bytes   
# processed_cloth = result

processed_user = human_model.process(user_image)


In [6]:
# Try on

 try_on = try_on_model.batch_try_on(processed_user, clothes=clothes)

# bc.bytes_to_image(try_on).save("/usr/src/app/volume/tmp/try_on_lower.jpg")
#  python3 -m app.pkg.ml.usage_examples.try_on 

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
100%|██████████| 20/20 [00:36<00:00,  1.80s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 768.00 MiB (GPU 0; 15.57 GiB total capacity; 13.61 GiB already allocated; 669.38 MiB free; 14.78 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [7]:
# bc.bytes_to_image(try_on[3])

In [8]:

# user_image = Image.open(human_fp)
# cloth_image = Image.open(cloth_fp)


# processed_user = human_model.process(user_image)

# no_background_image = cloth_model.model_background(cloth_image)
# no_background_image_bytes = bc.image_to_bytes(no_background_image)
# result = {}
# result["cloth"] = no_background_image_bytes   
# processed_cloth = result

# # Try on
# processed_user.update(
# {
#     "category": "upper_body",
#     # "cloth": no_background_image_bytes,
# }
# )


In [9]:
# try_on = try_on_model(processed_user)